# Set up

## MongoDB

In [11]:
!echo "[Check OS]"
!cat /etc/os-release
!echo "[Download MongoDB]"
!apt-get install mongo
!echo "[Install PyMongo]"
!pip install pymongo
!echo "[Start MongoDB]"
!service mongodb start

[Check OS]
NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
[Download MongoDB]
--2022-04-16 05:00:39--  https://fastdl.mongodb.org/linux/mongodb-linux-x86_64-ubuntu1804-5.0.7.tgz
Resolving fastdl.mongodb.org (fastdl.mongodb.org)... 108.157.162.116, 108.157.162.43, 108.157.162.48, ...
Connecting to fastdl.mongodb.org (fastdl.mongodb.org)|108.157.162.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86166297 (82M) [application/gzip]
Saving to: ‘mongodb-linux-x86_64-ubuntu1804-5.0.7.tgz’

mongodb-linux-x86_6 100%[===================>]  82.17M   108MB/s    in 0.8s    

2022-04-16 05:00:40 (108 MB/s) - ‘mongodb-linux-x86_64-ubuntu18

## PySpark

In [ ]:
!echo "[Download & Install OpenJDK]"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!echo "[Download Spark]"
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!echo "[Install Spark]"
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!echo "[Install findspark, pyspark]"
!pip install -q findspark
!pip install pyspark

# Download Data

In [2]:
!wget https://www.w3resource.com/mongodb-exercises/restaurants.zip
!unzip restaurants.zip
!mongoimport --db local --collection data --file restaurants.json

--2022-04-16 03:46:46--  https://www.w3resource.com/mongodb-exercises/restaurants.zip
Resolving www.w3resource.com (www.w3resource.com)... 172.67.69.127, 104.26.14.93, 104.26.15.93, ...
Connecting to www.w3resource.com (www.w3resource.com)|172.67.69.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262722 (257K) [application/zip]
Saving to: ‘restaurants.zip’

restaurants.zip     100%[===================>] 256.56K  --.-KB/s    in 0.03s   

2022-04-16 03:46:47 (7.71 MB/s) - ‘restaurants.zip’ saved [262722/262722]

Archive:  restaurants.zip
  inflating: restaurants.json        
2022-04-16T03:46:47.244+0000	connected to: localhost
2022-04-16T03:46:47.481+0000	imported 3772 documents


# Code

Exercise
https://www.w3resource.com/mongodb-exercises/#PracticeOnline

In [3]:
from pymongo import MongoClient
client = MongoClient()
print(f'DB: {client.list_database_names()}')
print('Collections: {}'.format([coll['name'] for coll in [*client['local'].list_collections()]]))
db = client['local']

DB: ['admin', 'local']
Collections: ['startup_log', 'data']


1.Write a MongoDB query to display the fields restaurant_id, name, borough and cuisine for all the documents in the collection restaurant

In [4]:
db['data'].find_one()

{'_id': ObjectId('625a3c2716f0cf4ac6d100d0'),
 'address': {'building': '1007',
  'coord': [-73.856077, 40.848447],
  'street': 'Morris Park Ave',
  'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6},
  {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10},
  {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9},
  {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}

2.Write a MongoDB query to display the fields restaurant_id, name, borough and cuisine for all the documents in the collection restaurant

In [5]:
[*db.data.aggregate([
    {'$project': {'restaurant_id': 1
                  , 'name': 1
                  , 'borough': 1
                  , 'cuisine': 1}}
    , {'$limit': 3}
])]

[{'_id': ObjectId('625a3c2716f0cf4ac6d100d0'),
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d100d1'),
  'borough': 'Brooklyn',
  'cuisine': 'Hamburgers',
  'name': "Wendy'S",
  'restaurant_id': '30112340'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d100d2'),
  'borough': 'Manhattan',
  'cuisine': 'Irish',
  'name': 'Dj Reynolds Pub And Restaurant',
  'restaurant_id': '30191841'}]

3. Write a MongoDB query to display the fields restaurant_id, name, borough and cuisine, but exclude the field _id for all the documents in the collection restaurant

In [6]:
[*db.data.aggregate([
    {'$project': {'restaurant_id': 1
                  , 'name': 1, 'borough': 1
                  , 'cuisine': 1
                  , '_id': 0}}
    , {'$limit': 3}
])]

[{'borough': 'Bronx',
  'cuisine': 'Bakery',
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'borough': 'Brooklyn',
  'cuisine': 'Hamburgers',
  'name': "Wendy'S",
  'restaurant_id': '30112340'},
 {'borough': 'Manhattan',
  'cuisine': 'Irish',
  'name': 'Dj Reynolds Pub And Restaurant',
  'restaurant_id': '30191841'}]

4. Write a MongoDB query to display the fields restaurant_id, name, borough and zip code, but exclude the field _id for all the documents in the collection restaurant.

In [7]:
[*db.data.aggregate([
  {'$project': {'restaurant_id' : 1
                , 'name': 1
                , 'borough': 1
                , 'address.zipcode': 1
                , '_id': 0}}
  , {'$limit': 3}
])]

[{'address': {'zipcode': '10462'},
  'borough': 'Bronx',
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'address': {'zipcode': '11225'},
  'borough': 'Brooklyn',
  'name': "Wendy'S",
  'restaurant_id': '30112340'},
 {'address': {'zipcode': '10019'},
  'borough': 'Manhattan',
  'name': 'Dj Reynolds Pub And Restaurant',
  'restaurant_id': '30191841'}]

5. Write a MongoDB query to display all the restaurant which is in the borough Bronx.

In [8]:
[*db.data.aggregate([
  {'$match': {'borough': 'Bronx'}}
  , {'$project': {'restaurant_id' : 1
                , 'name': 1
                , 'borough': 1
                , 'address.zipcode': 1
                , '_id': 0}}
  , {'$limit': 3}
])]

[{'address': {'zipcode': '10462'},
  'borough': 'Bronx',
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'address': {'zipcode': '10460'},
  'borough': 'Bronx',
  'name': 'Wild Asia',
  'restaurant_id': '40357217'},
 {'address': {'zipcode': '10466'},
  'borough': 'Bronx',
  'name': 'Carvel Ice Cream',
  'restaurant_id': '40363093'}]

6. Write a MongoDB query to display the first 5 restaurant which is in the borough Bronx

In [9]:
[*db.data.aggregate([
  {'$match': {'borough': 'Bronx'}}
  , {'$project': {'restaurant_id' : 1
                , 'name': 1
                , 'borough': 1
                , 'address.zipcode': 1
                , '_id': 0}}
  , {'$limit': 5}
])]

[{'address': {'zipcode': '10462'},
  'borough': 'Bronx',
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'address': {'zipcode': '10460'},
  'borough': 'Bronx',
  'name': 'Wild Asia',
  'restaurant_id': '40357217'},
 {'address': {'zipcode': '10466'},
  'borough': 'Bronx',
  'name': 'Carvel Ice Cream',
  'restaurant_id': '40363093'},
 {'address': {'zipcode': '10474'},
  'borough': 'Bronx',
  'name': 'Happy Garden',
  'restaurant_id': '40363289'},
 {'address': {'zipcode': '10458'},
  'borough': 'Bronx',
  'name': 'Happy Garden',
  'restaurant_id': '40364296'}]

7. Write a MongoDB query to display the next 5 restaurants after skipping first 5 which are in the borough Bronx.

In [10]:
[*db.data.aggregate([
  {'$match': {'borough': 'Bronx'}}
  , {'$project': {'restaurant_id' : 1
                , 'name': 1
                , 'borough': 1
                , 'address.zipcode': 1
                , '_id': 0}}
  , {'$skip': 5}
  , {'$limit': 5}
])]

[{'address': {'zipcode': '10465'},
  'borough': 'Bronx',
  'name': 'Manhem Club',
  'restaurant_id': '40364363'},
 {'address': {'zipcode': '10462'},
  'borough': 'Bronx',
  'name': 'The New Starling Athletic Club Of The Bronx',
  'restaurant_id': '40364956'},
 {'address': {'zipcode': '10451'},
  'borough': 'Bronx',
  'name': 'Yankee Tavern',
  'restaurant_id': '40365499'},
 {'address': {'zipcode': '10467'},
  'borough': 'Bronx',
  'name': 'Mcdwyers Pub',
  'restaurant_id': '40365893'},
 {'address': {'zipcode': '10463'},
  'borough': 'Bronx',
  'name': 'The Punch Bowl',
  'restaurant_id': '40366497'}]

8. Write a MongoDB query to find the restaurants who achieved a score more than 90. 

In [11]:
[*db.data.aggregate([
  {'$match': {'grades.score': {'$gt': 90}}}
  , {'$limit': 2}
])]

[{'_id': ObjectId('625a3c2716f0cf4ac6d1022e'),
  'address': {'building': '65',
   'coord': [-73.9782725, 40.7624022],
   'street': 'West   54 Street',
   'zipcode': '10019'},
  'borough': 'Manhattan',
  'cuisine': 'American ',
  'grades': [{'date': datetime.datetime(2014, 8, 22, 0, 0),
    'grade': 'A',
    'score': 11},
   {'date': datetime.datetime(2014, 3, 28, 0, 0), 'grade': 'C', 'score': 131},
   {'date': datetime.datetime(2013, 9, 25, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2013, 4, 8, 0, 0), 'grade': 'B', 'score': 25},
   {'date': datetime.datetime(2012, 10, 15, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2011, 10, 19, 0, 0), 'grade': 'A', 'score': 13}],
  'name': "Murals On 54/Randolphs'S",
  'restaurant_id': '40372466'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d102d0'),
  'address': {'building': '345',
   'coord': [-73.9864626, 40.7266739],
   'street': 'East 6 Street',
   'zipcode': '10003'},
  'borough': 'Manhattan',
  'cuisine': 'I

9. Write a MongoDB query to find the restaurants that achieved a score, more than 80 but less than 100.

In [12]:
[*db.data.aggregate([
  {'$match': {'grades.score': {'$gt': 80}}}
  , {'$match': {'grades.score': {'$lt': 100}}}
  , {'$limit': 2}
])]

[{'_id': ObjectId('625a3c2716f0cf4ac6d1022e'),
  'address': {'building': '65',
   'coord': [-73.9782725, 40.7624022],
   'street': 'West   54 Street',
   'zipcode': '10019'},
  'borough': 'Manhattan',
  'cuisine': 'American ',
  'grades': [{'date': datetime.datetime(2014, 8, 22, 0, 0),
    'grade': 'A',
    'score': 11},
   {'date': datetime.datetime(2014, 3, 28, 0, 0), 'grade': 'C', 'score': 131},
   {'date': datetime.datetime(2013, 9, 25, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2013, 4, 8, 0, 0), 'grade': 'B', 'score': 25},
   {'date': datetime.datetime(2012, 10, 15, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2011, 10, 19, 0, 0), 'grade': 'A', 'score': 13}],
  'name': "Murals On 54/Randolphs'S",
  'restaurant_id': '40372466'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d102d0'),
  'address': {'building': '345',
   'coord': [-73.9864626, 40.7266739],
   'street': 'East 6 Street',
   'zipcode': '10003'},
  'borough': 'Manhattan',
  'cuisine': 'I

10. Write a MongoDB query to find the restaurants which locate in latitude value less than -95.754168

In [21]:
[*db.data.aggregate([
    {'$match': {'address.coord.0': {'$lt': -95.754168}}}
    , {'$limit': 2}
])]

[{'_id': ObjectId('625a3c2716f0cf4ac6d10718'),
  'address': {'building': '3707',
   'coord': [-101.8945214, 33.5197474],
   'street': '82 Street',
   'zipcode': '11372'},
  'borough': 'Queens',
  'cuisine': 'American ',
  'grades': [{'date': datetime.datetime(2014, 6, 4, 0, 0),
    'grade': 'A',
    'score': 12},
   {'date': datetime.datetime(2013, 11, 7, 0, 0), 'grade': 'B', 'score': 19},
   {'date': datetime.datetime(2013, 5, 17, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2012, 8, 29, 0, 0), 'grade': 'A', 'score': 11},
   {'date': datetime.datetime(2012, 4, 3, 0, 0), 'grade': 'A', 'score': 12},
   {'date': datetime.datetime(2011, 11, 16, 0, 0), 'grade': 'A', 'score': 7}],
  'name': 'Burger King',
  'restaurant_id': '40534067'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d10a85'),
  'address': {'building': '15259',
   'coord': [-119.6368672, 36.2504996],
   'street': '10 Avenue',
   'zipcode': '11357'},
  'borough': 'Queens',
  'cuisine': 'Italian',
  'grades': [{'date

30. Write a MongoDB query which will select the restaurant Id, name and grades for those restaurants which returns 0 as a remainder after dividing the score by 7

In [52]:
[*db.data.aggregate([
    {'$unwind': '$grades'}
    , {'$group': {'_id': '$restaurant_id', 'sumScore' : {'$sum': '$grades.score'}}}
    , {'$match': {'sumScore': {'$mod': [7, 0]}}}
    , {'$lookup': {
        'from': 'data'
        , 'localField': '_id'
        , 'foreignField': 'restaurant_id'
        , 'as': 'restaurant_detail_info'
    }}
    , {'$project': {
        '_id': 0
        , 'sumScore': 1
        , 'restaurant_detail_info.name': 1
        , 'restaurant_detail_info.borough': 1
        , 'restaurant_detail_info.cuisine': 1
    }}
    , {'$limit': 3}
])]

[{'restaurant_detail_info': [{'borough': 'Brooklyn',
    'cuisine': 'Chinese',
    'name': 'Chopstix Restaurant'}],
  'sumScore': 91},
 {'restaurant_detail_info': [{'borough': 'Manhattan',
    'cuisine': 'Sandwiches/Salads/Mixed Buffet',
    'name': 'Europa Cafe'}],
  'sumScore': 42},
 {'restaurant_detail_info': [{'borough': 'Manhattan',
    'cuisine': 'American ',
    'name': 'Tsq Brasserie'}],
  'sumScore': 42}]

31. Write a MongoDB query to find the restaurant name, borough, longitude and attitude and cuisine for those restaurants which contains 'mon' as three letters somewhere in its name.

In [27]:
[*db.data.aggregate([
    {'$match': {'name': {'$regex': 'mon'}}}
    , {'$project': {
        'name': 1
        , 'borough': 1
        , 'address.coord': 1
        , 'cuisine': 1
        , '_id': 0
    }}
    , {'$limit': 3}
])]

[{'address': {'coord': [-73.98306099999999, 40.7441419]},
  'borough': 'Manhattan',
  'cuisine': 'American ',
  'name': "Desmond'S Tavern"},
 {'address': {'coord': [-73.8221418, 40.7272376]},
  'borough': 'Queens',
  'cuisine': 'Jewish/Kosher',
  'name': 'Shimons Kosher Pizza'},
 {'address': {'coord': [-74.10465599999999, 40.58834]},
  'borough': 'Staten Island',
  'cuisine': 'American ',
  'name': 'Richmond County Country Club'}]

32. Write a MongoDB query to find the restaurant name, borough, longitude and latitude and cuisine for those restaurants which contain 'Mad' as first three letters of its name. 

In [26]:
[*db.data.aggregate([
    {'$match': {'name': {'$regex': '^Mad'}}}
    , {'$project': {
        'name': 1
        , 'borough': 1
        , 'address.coord': 1
        , 'cuisine': 1
        , '_id': 0
    }}
    , {'$limit': 3}
])]

[{'address': {'coord': [-73.9860597, 40.7431194]},
  'borough': 'Manhattan',
  'cuisine': 'American ',
  'name': 'Madison Square'},
 {'address': {'coord': [-73.98302199999999, 40.742313]},
  'borough': 'Manhattan',
  'cuisine': 'Indian',
  'name': 'Madras Mahal'},
 {'address': {'coord': [-74.000002, 40.72735]},
  'borough': 'Manhattan',
  'cuisine': 'American ',
  'name': 'Madame X'}]

Extra 1: Moving Average

In [ ]:
db.stockPrices.aggregate( [
   {
      $setWindowFields: {
         partitionBy: "$stock",
         sortBy: { date: 1 },
         output: {
            expMovingAvgForStock: {
               $expMovingAvg: { input: "$price", N: 2 }
            }
         }
      }
   }
] )

In [56]:
[*db.data.aggregate([
    {'$unwind': '$grades'}
    , {'$setWindowFields': {
        'partitionBy': '$restaurant_id'
        , 'sortBy': '$grades.date'
        , 'output': {
            'movingAvgGrade': {
                '$expMovingAvg': {
                    'input': '$grades.score'
                    , 'N': 3
                }
            }
        }
    }}
])]

OperationFailure: ignored

In [54]:
[*db.data.aggregate([
    {'$unwind': '$grades'}
    , {'$limit': 3}
])]

[{'_id': ObjectId('625a3c2716f0cf4ac6d100d0'),
  'address': {'building': '1007',
   'coord': [-73.856077, 40.848447],
   'street': 'Morris Park Ave',
   'zipcode': '10462'},
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'grades': {'date': datetime.datetime(2014, 3, 3, 0, 0),
   'grade': 'A',
   'score': 2},
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d100d0'),
  'address': {'building': '1007',
   'coord': [-73.856077, 40.848447],
   'street': 'Morris Park Ave',
   'zipcode': '10462'},
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'grades': {'date': datetime.datetime(2013, 9, 11, 0, 0),
   'grade': 'A',
   'score': 6},
  'name': 'Morris Park Bake Shop',
  'restaurant_id': '30075445'},
 {'_id': ObjectId('625a3c2716f0cf4ac6d100d0'),
  'address': {'building': '1007',
   'coord': [-73.856077, 40.848447],
   'street': 'Morris Park Ave',
   'zipcode': '10462'},
  'borough': 'Bronx',
  'cuisine': 'Bakery',
  'grades': {'date': datet